In [4]:
import visualkeras
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow.keras.applications as LayerApp
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
import tensorflow as tf
from tensorflow.keras.callbacks import Callback

from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.applications import ResNet50
import os
import time
import pickle
import numpy as np
import absl.logging

class CustomCNN(Model):
    def __init__(self, num_classes=2):
        super(CustomCNN, self).__init__()
        # Add L2 regularization to the convolutional layers
        #self.conv1 = Conv1D(32, 3, activation='relu', padding='same', kernel_regularizer=l2(0.00001))
        self.conv1 = Conv1D(32, 3, activation='relu', padding='same')
        #self.bn1 = BatchNormalization()
        self.pool1 = MaxPooling1D(2)
        #self.dropout1 = Dropout(0.2)

        #self.conv2 = Conv1D(64, 3, activation='relu', padding='same', kernel_regularizer=l2(0.00001))
        self.conv2 = Conv1D(64, 3, activation='relu', padding='same')
        #self.bn2 = BatchNormalization()
        #self.pool2 = MaxPooling1D(2)
        #self.dropout2 = Dropout(0.2)

        self.flatten = Flatten()
        self.dense1 = Dense(256, activation='relu')
        self.dropout3 = Dropout(0.2)
        self.dense2 = Dense(256, activation='relu')
        self.dense3 = Dense(num_classes, activation='softmax')

    def call(self, inputs, training=False):
        # 시퀀스 길이 대신 특징 차원에 대한 차원 추가
        inputs = tf.expand_dims(inputs, axis=-1)
        x = self.conv1(inputs)
        #x = self.bn1(x, training=training)
        x = self.pool1(x)
        #x = self.dropout1(x, training=training)

        x = self.conv2(x)
        #x = self.bn2(x, training=training)
        #x = self.pool2(x)
        #x = self.dropout2(x, training=training)

        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dropout3(x, training=training)
        x = self.dense2(x)
        return self.dense3(x)

class FNN(Model):
    def __init__(self, num_classes=2):
        super(FNN, self).__init__()
        self.dense1 = Dense(256, activation='relu')
        self.dense2 = Dense(256, activation='relu')
        self.dense3 = Dense(256, activation='relu')
        self.dense_last = Dense(num_classes, activation='softmax')

    def call(self, inputs, training=False):
        # 시퀀스 길이 대신 특징 차원에 대한 차원 추가
        x = self.dense1(inputs)
        x = self.dense2(x)
        x = self.dense3(x)
        return self.dense_last(x)


def model_fnn(input_witdh):
    model = FNN()
    model.build((None, input_witdh))
    return model

def model_custom(input_width):
    model = CustomCNN()
    model.build((None, input_width))
    return model

def model_resnet(input_width,layer_depth):
    # 사전 훈련된 EfficientNetB2 모델 로드
    base_model = ResNet50(include_top=False, weights="imagenet", input_shape=(input_width, input_width, 3))

    total_layers = len(base_model.layers)
    trainable_layers = int(total_layers * layer_depth)  # 하위 j만 훈련

    for layer in base_model.layers[:-trainable_layers]:
        layer.trainable = False

    # 커스텀 층 추가
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = BatchNormalization()(x)  # 배치 정규화 추가
    x = Dropout(0.2)(x)  # 드롭아웃 추가

    predictions = Dense(2, activation='softmax')(x)  # 다중 클래스 분류를 위한 설정

    # 새로운 모델 정의
    model = Model(inputs=base_model.input, outputs=predictions)
    return model


def model_efficientnet(input_width,layer_depth):
    # 사전 훈련된 EfficientNetB2 모델 로드
    base_model = LayerApp.EfficientNetV2M(include_top=False, weights="imagenet", input_shape=(input_width, input_width, 3))

    total_layers = len(base_model.layers)
    trainable_layers = int(total_layers * layer_depth)  # 전체가 1일 때, layer_depth만큼 ratio에 대해 학습

    for layer in base_model.layers[:-trainable_layers]:
        layer.trainable = False

    # 커스텀 층 추가
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = BatchNormalization()(x)  # 배치 정규화 추가
    x = Dropout(0.4)(x)  # 드롭아웃 추가

    predictions = Dense(2, activation='softmax')(x)  # 다중 클래스 분류를 위한 설정

    # 새로운 모델 정의
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

model1 = FNN()
model1.summary()
# visualkeras.layered_view(model1)

AttributeError: 'function' object has no attribute 'summary'